In [13]:
#for handling data
import numpy as np
import pandas as pd

#for scraping
import requests
from bs4 import BeautifulSoup
import re
import json
import string

#for concurrency
from queue import Queue, SimpleQueue
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import threading

### Strategy for scraping

- All of the required data is contained in a script tag which begins with the text "window.__INITIAL_STATE__"

- First I have extracted all script tags from the soup

- Then I have extracted the required script tag (by checking if it begins with the "window.__INITIAL_STATE__" text)

- Then I perform a bit of cleaning before loading the required data as json

Note - Each of the script tags only contain 20 of the items even though we see 30 items per page. But the remaining items are available by making requests to the next page 

### Logic 
For each label keep making requests for each page till you don't get a status 200 code (this does not seem to work, the pages which do not have any data also get a success status code)



In [2]:
def extract_json(soup):
    all_scripts=soup.find_all("script")
    
    #extracting the script tag as a string (storing in a list)
    all_med_data=[]
    for script in all_scripts:
        # to check if a script tag can be converted to a string and if the string has the "window.__INITIAL__STATE__"
        if (script.string) and ("window.__INITIAL_STATE__" in script.string):
            all_med_data.append(script.string)
    json_data=all_med_data[0].replace("\n    window.__INITIAL_STATE__ = ",""
                       ).replace(";\n    window.__STATUS_CODE__ = null;\n","")
    json_data=json.loads(json_data)
    
    return(json_data['allMedicinePageReducer']['data'])

%%time
base_url="https://www.1mg.com/drugs-all-medicines"
label=list(string.ascii_lowercase)
page=list(range(1,1000))
results=[]
for i in page:
    for j in ['a','b']:
        response=requests.get(f'{base_url}?page={str(i)}&label={str(j)}')
        soup=BeautifulSoup(response.content, "html.parser")
        if (response.status_code==200) and (extract_json(soup)['skus']):
            results.append(extract_json(BeautifulSoup(response.content, "html.parser")))
        else:
            break;

### Trying to implement the Producer/Consumer model for Threading using concurrent.futures

In [3]:
tasks=Queue()
results=SimpleQueue()

In [6]:
# resp=requests.get("https://www.1mg.com/drugs-all-medicines?page=1&label=b")
# resp.text

'<!DOCTYPE html>\n<html lang="en">\n    <head>\n        <meta charset="utf-8" />\n        <meta\n            name="viewport"\n            content="initial-scale=1, maximum-scale=1, user-scalable=no, width=device-width height=device-height"\n        />\n        <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\n        <meta http-equiv="x-dns-prefetch-control" content="on" />\n\n        <meta name="theme-color" content="#FFF3E3" />\n        <!-- Windows Phone -->\n        <meta name="msapplication-navbutton-color" content="#FFF3E3" />\n        <!-- iOS Safari -->\n        <meta name="apple-mobile-web-app-status-bar-style" content="#FFF3E3" />\n        <meta name="apple-mobile-web-app-capable" content="yes" />\n\n        \n\n        <!-- Dynamic seo tags -->\n        \n        <title>Get List of All Medicines Starting with Alphabet B at Single Place.</title>\n        <meta name="keywords" content="undefined" data-react-helmet="true"/>\n        <meta property="og:title" con

In [7]:
#creating tasks
base_url="https://www.1mg.com/drugs-all-medicines"
labels=list(string.ascii_lowercase)
page=list(range(1,2000))
for i in labels:
    for j in page:
        tasks.put(f'{base_url}?page={str(j)}&label={str(i)}')

In [11]:
tasks.get()

'https://www.1mg.com/drugs-all-medicines?page=4&label=a'

In [17]:
dir(tasks)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get',
 '_init',
 '_put',
 '_qsize',
 'all_tasks_done',
 'empty',
 'full',
 'get',
 'get_nowait',
 'join',
 'maxsize',
 'mutex',
 'not_empty',
 'not_full',
 'put',
 'put_nowait',
 'qsize',
 'queue',
 'task_done',
 'unfinished_tasks']

In [ ]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Save-Data': 'on',
    'TE': 'Trailers',
    }
# resp=requests.get("https://www.1mg.com/drugs-all-medicines?page=1&label=b", headers=HEADERS)
# resp.text
thread_local = threading.local()

def get_session():
    if not hasattr(thread_local, 'session'):
        thread_local.session = requests.Session()
    return thread_local.session

In [60]:

def scraping_worker(tasks,results):
    j=0
    session = get_session()
    while True:
        try:
            task = tasks.get(block=False) 
        except queue.Empty: #
            print('Queue is empty! My work here is done. Exiting.')
            return
        tasks.task_done()
        response=session.get(task)
        soup=BeautifulSoup(response.content, "html.parser")
#         print(response)
        if (response.status_code==200) and (extract_json(soup)['skus']):
            results.put(extract_json(BeautifulSoup(response.content, "html.parser")))
            j+=1
            print(f"result {j} stored")
        

In [61]:
%%time
with ThreadPoolExecutor(max_workers=15) as ex:
    futures = [
        ex.submit(scraping_worker, tasks, results) for _ in range(15)
    ]

<Response [200]>
result 1 stored
<Response [200]>
result 1 stored
<Response [200]>
result 1 stored
<Response [200]>
<Response [200]>
result 1 stored
result 1 stored
<Response [200]>
result 1 stored
<Response [200]>
<Response [200]>
result 1 stored
result 1 stored
<Response [200]>
<Response [200]>
result 1 stored
<Response [200]>
result 1 stored
result 1 stored
<Response [200]>
result 1 stored
<Response [200]>
result 2 stored
<Response [200]>
result 2 stored
<Response [200]>
result 2 stored
<Response [200]>
result 2 stored
<Response [200]>
result 2 stored
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
result 2 stored
result 2 stored
result 2 stored
<Response [200]>
result 2 stored
result 2 stored
<Response [200]>
<Response [200]>
result 3 stored
result 3 stored<Response [200]>

result 2 stored
<Response [200]>
result 3 stored<Response [200]>

result 2 stored
<Response [200]>
result 3 stored
<Response [200]>
result 3 stored
<Response [200]>
result 3 stored
<Response 

result 22 stored
<Response [200]><Response [200]>

result 21 stored
result 20 stored<Response [200]>
result 20 stored

<Response [200]><Response [200]>

<Response [200]>
result 22 storedresult 21 storedresult 21 stored


<Response [200]>
<Response [200]><Response [200]>
result 22 stored
result 21 stored
result 23 stored

<Response [200]>
result 22 stored
<Response [200]>
<Response [200]>result 23 stored

<Response [200]>
result 23 stored
result 21 stored
<Response [200]>
result 21 stored
<Response [200]>
<Response [200]>
result 22 stored
result 22 stored
<Response [200]>
result 23 stored
<Response [200]>
<Response [200]>result 23 stored
<Response [200]>

result 22 stored
result 24 stored
<Response [200]><Response [200]>
result 23 stored

result 22 stored
<Response [200]>
<Response [200]>
result 22 stored
result 24 stored
<Response [200]>
result 22 stored
<Response [200]>
result 24 stored
<Response [200]>
result 24 stored
<Response [200]>
result 23 stored
<Response [200]>
result 23 stor

result 41 stored
<Response [200]><Response [200]><Response [200]>

result 41 stored

result 42 storedresult 40 stored

<Response [200]>
result 42 stored
<Response [200]>
<Response [200]>result 41 stored
<Response [200]>

<Response [200]><Response [200]>result 43 stored

<Response [200]>result 41 stored

<Response [200]>

result 42 storedresult 43 stored

result 42 storedresult 42 stored

<Response [200]>
result 43 stored
<Response [200]>
result 43 stored
<Response [200]>
result 43 stored
<Response [200]>
<Response [200]>
result 42 stored
result 42 stored
<Response [200]><Response [200]>

<Response [200]>result 44 stored
result 41 stored

result 42 stored
<Response [200]><Response [200]>
result 43 stored
result 43 stored
<Response [200]>
result 44 stored

<Response [200]>
result 43 stored
<Response [200]>
result 44 stored
<Response [200]>
result 44 stored
<Response [200]>
result 44 stored
<Response [200]>
<Response [200]>
result 42 storedresult 43 stored
<Response [200]>

result 44 stor

<Response [200]>
<Response [200]>
result 62 stored
result 61 stored
<Response [200]>
<Response [200]>
result 62 stored
result 62 stored
<Response [200]>
<Response [200]>
result 61 stored
<Response [200]>
result 63 stored
result 62 stored<Response [200]>

<Response [200]>
result 60 stored
result 62 stored
<Response [200]>
result 64 stored
<Response [200]>
result 63 stored
<Response [200]>
<Response [200]>
result 63 stored
result 61 stored
<Response [200]><Response [200]>
result 63 stored

result 62 stored
<Response [200]>
<Response [200]>
result 64 stored
result 63 stored
<Response [200]>
result 63 stored
<Response [200]>
result 65 stored
<Response [200]>
result 61 stored
<Response [200]>
result 63 stored
<Response [200]>
result 62 stored
<Response [200]>
result 62 stored
<Response [200]>
result 64 stored
<Response [200]>
result 64 stored
<Response [200]>
result 65 stored
<Response [200]>
result 63 stored
<Response [200]>
<Response [200]>
result 64 stored
<Response [200]>
result 64 stor

<Response [200]>
result 83 stored
<Response [200]>
result 82 stored
<Response [200]>
result 83 stored
<Response [200]>
result 81 stored
<Response [200]>
result 78 stored
<Response [200]>
result 81 stored
<Response [200]><Response [200]>

result 82 stored
<Response [200]>
result 83 stored
result 84 stored
<Response [200]><Response [200]>

result 83 storedresult 84 stored

<Response [200]>
<Response [200]>
result 85 stored
result 84 stored
<Response [200]>
<Response [200]>
result 84 stored
result 83 stored
<Response [200]>
result 82 stored
<Response [200]>
result 79 stored
<Response [200]>
<Response [200]>
result 82 stored
result 83 stored
<Response [200]>
result 84 stored
<Response [200]>
result 85 stored
<Response [200]>
result 84 stored
<Response [200]>
<Response [200]>
result 84 stored
<Response [200]>
result 85 storedresult 86 stored

<Response [200]>
<Response [200]>
result 85 stored
result 83 stored
<Response [200]>
result 80 stored
<Response [200]>
result 85 stored
<Response [200

...

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]><Response [403]>

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]><Response [403]>

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]><Response [403]>

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]><Response [403]>

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]><Response [403]>

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
Wall time: 12min 11s


In [62]:
all([f.done() for f in futures])

True

In [67]:
tasks.qsize()

0

In [63]:
results.qsize()

2374

In [65]:
results

### Stuff to record for each card - 

1. Id and Skuid (for future fetching of data)

2. Name

3. Prescription req
    
4. is_discontinued

5. manufacturer

6. Type

7. Pack size

8. short_composition

9. rx_required.header

In [66]:
a=results.get()
a

{'skus': [{'is_discontinued': False,
   'manufacturer_name': 'Cipla Ltd',
   'type': 'allopathy',
   'price': 271.52,
   'name': 'Alfusin Tablet PR',
   'id': 41805,
   'sku_id': 41805,
   'available': True,
   'pack_size_label': 'strip of 15 Tablet pr',
   'rx_required': {'header': 'Prescription Required',
    'icon_url': 'https://onemg.gumlet.io/image/upload/w_20,h_20/q_auto,f_auto/rx_icon.png'},
   'slug': '/drugs/alfusin-tablet-pr-41805',
   'short_composition': 'Alfuzosin (10mg)',
   'image_url': 'https://onemg.gumlet.io/image/upload/a_ignore,w_380,h_380,c_fit,q_auto,f_auto/v1537758726/bdv6ssvviagsmrj57ucq.jpg',
   'in_stock': {'add': {'action_label': 'Adding..',
     'final_label': 'ADDED',
     'label': 'ADD'},
    'cart_actions': {'dweb': ['add'], 'mweb': ['add']}},
   'quantity': 15},
  {'is_discontinued': False,
   'manufacturer_name': 'Aurz Pharmaceuticals Pvt Ltd',
   'type': 'allopathy',
   'price': 69.8,
   'name': 'Axcet M Tablet',
   'id': 234101,
   'sku_id': 234101,
 

In [99]:
final_df=pd.DataFrame()
for i in range(0,results.qsize()):
    result=results.get()
    temp_df=pd.DataFrame([[i.get('sku_id'),
                   i.get('name'),
                   i.get('manufacturer_name'),
                   i.get('type'),
                   i.get('pack_size_label'),
                   i.get('price'),
                   i.get('rx_required'),
                   i.get('short_composition'),
                   i.get('is_discontinued')] for i in result['skus']],
                 columns=['sku_id',
                 'name','manufacturer_name',
                 'type','pack_size_label',
                 'price','rx_required',
                 'short_composition','is_discontinued'])
    final_df=pd.concat([final_df,temp_df],axis='rows',ignore_index=True)
    

In [107]:
final_df['name'].sort_values()

3312                   A 1 5mg Tablet
5929                 A 250 Suspension
10016        A 3 100 mg/500 mg Tablet
10989           A Arti 60mg Injection
10035      A Arti L 80mg/480mg Tablet
                     ...             
46685               Hyzer 10mg Tablet
46618              Hyzine 25mg Tablet
46873                Hyzix 10mg Syrup
47172    Hyzol D 30mg/40mg Capsule SR
47210            Hyzolid 600mg Tablet
Name: name, Length: 47216, dtype: object

In [112]:
final_df.to_csv("drugs.csv",index=False)

In [19]:
a=requests.get("https://www.1mg.com/search/all?name=orid")

BeautifulSoup(a.content,"html.parser")

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="initial-scale=1, maximum-scale=1, user-scalable=no, width=device-width height=device-height" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<meta content="#FFF3E3" name="theme-color"/>
<!-- Windows Phone -->
<meta content="#FFF3E3" name="msapplication-navbutton-color"/>
<!-- iOS Safari -->
<meta content="#FFF3E3" name="apple-mobile-web-app-status-bar-style"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<!-- Dynamic seo tags -->
<title>Online Pharmacy India | Buy Medicines from India's Trusted Medicine Store: 1mg.com</title>
<meta content="online pharmacy, online medicine, medicine online, online medical store, buy medicine online, online pharmacy India, buy medicine online India, online chemist, online medicine purchase" data-react-helmet="true" name="keywords"/>
<meta content="Online Pharmacy India | B